# 이분 그래프 기반 추천 시스템 테스트

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch_geometric.data import Data

# 예시 유저 데이터
interactions = pd.DataFrame(columns=['user_id', 'movie_id'])
interactions['user_id'] = [1, 1, 1, 2, 2]
interactions['movie_id'] = [3, 4, 5951, 5459, 6076]
# interactions['user_id'] = [1, 1]
# interactions['movie_id'] = [5459, 6076]  # 인 타임, 점퍼
users = pd.DataFrame(columns=['user_id'])
users['user_id'] = interactions.user_id.unique()

num_users = users.shape[0]
num_movies = 9525
num_in_features = 406
num_out_features = 128  # Hidden dimension 크기 조정
num_of_heads = 1

# 영화 features
movie_features = torch.from_numpy(np.load("final_features_without_directors_0511.npy")).float()

# user indices, movie indices 생성
user_id_to_index = {user_id: idx for idx, user_id in enumerate(users['user_id'])}
movie_id_to_index = {movie_id: idx for idx, movie_id in enumerate(range(num_movies))}

# 유저 - 영화 간 상호작용 edge index로 변환
user_indices = interactions['user_id'].apply(lambda x: user_id_to_index[x])
movie_indices = interactions['movie_id'].apply(lambda x: num_users + movie_id_to_index[x])  # 유저 수만큼 offset 추가

edge_index = torch.tensor([user_indices.values, movie_indices.values], dtype=torch.long)

# 유저 초기 임베딩 생성
user_features = torch.zeros(num_users, num_in_features)
# 유저 초기 임베딩을 xavier 초기화
nn.init.xavier_uniform_(user_features)  # initialize user features with xavier initialization
# 영화 feature와 유저 feature 합치기
x = torch.cat([user_features, movie_features], dim=0)

data = Data(x=x, edge_index=edge_index)

# 유저, 영화의 임베딩 생성용 GAT
class GATLinkPredictor(nn.Module):
    def __init__(self, num_in_features, num_out_features=128, num_of_heads=1):
        super().__init__()
        self.conv1 = GATConv(num_in_features, num_out_features, heads=num_of_heads)
        self.conv2 = GATConv(num_out_features * num_of_heads, num_out_features, heads=num_of_heads)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        return x

# 유저 - 영화 간 edge를 입력받아 유저-영화 간 관계 예측
class LinkPredictor(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim * 2, 1)

    def forward(self, z, edge_index):
        row, col = edge_index
        z_row = z[row]
        z_col = z[col]
        z_concat = torch.cat([z_row, z_col], dim=1)
        return torch.sigmoid(self.linear(z_concat))

# 모델 초기화
gat_model = GATLinkPredictor(num_in_features, num_out_features, num_of_heads)
link_predictor = LinkPredictor(num_out_features * num_of_heads)

# Negative sampling
def negative_sampling(edge_index, num_users, num_movies, num_neg_samples):
    existing_edges = set(zip(edge_index[0].tolist(), edge_index[1].tolist()))
    neg_edge_index = set()
    
    while len(neg_edge_index) < num_neg_samples:
        i = torch.randint(0, num_users, (1,)).item()
        j = torch.randint(0, num_movies, (1,)).item() + num_users  # Add offset to movie indices
        if (i, j) not in existing_edges and (i, j) not in neg_edge_index:
            neg_edge_index.add((i, j))
    
    neg_edge_index = torch.tensor(list(neg_edge_index)).t()
    return neg_edge_index

# negative samples
num_neg_samples = edge_index.size(1)
neg_edge_index = negative_sampling(edge_index, num_users, num_movies, num_neg_samples)

# positive, negative samples 합치기
train_edge_index = torch.cat([edge_index, neg_edge_index], dim=1)
train_labels = torch.cat([torch.ones(edge_index.size(1)), torch.zeros(neg_edge_index.size(1))])

# Train 함수 정의
def train(model, predictor, data, train_edge_index, train_labels, optimizer, epochs=100):
    model.train()
    predictor.train()
    criterion = nn.BCELoss()

    for epoch in range(epochs):
        optimizer.zero_grad()
        node_embeddings = model(data.x, data.edge_index)
        scores = predictor(node_embeddings, train_edge_index)
        loss = criterion(scores.squeeze(), train_labels)
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

# Initialize optimizer
optimizer = torch.optim.Adam(list(gat_model.parameters()) + list(link_predictor.parameters()), lr=0.01)

# 모델 학습
train(gat_model, link_predictor, data, train_edge_index, train_labels, optimizer, epochs=100)

# Test

In [449]:
# Function to recommend top 5 movies for a given user
def recommend_movies_for_user(user_id, node_embeddings, edge_index, num_recommendations=5):
    user_idx = torch.tensor([user_id_to_index[user_id]])
    movie_indices = torch.arange(num_users, num_users + num_movies)
    
    # Generate all possible user-movie pairs
    pairs = torch.stack([user_idx.expand(num_movies), movie_indices], dim=0)
    
    # Check existing edges to avoid recommending already liked movies
    existing_edges = edge_index[:, (edge_index[0] == user_idx)]
    existing_movies = existing_edges[1] - num_users
    
    # Remove existing edges from pairs
    mask = torch.ones(num_movies, dtype=torch.bool)
    mask[existing_movies] = False
    pairs = pairs[:, mask]
    
    # Get scores for the user-movie pairs
    scores = link_predictor(node_embeddings, pairs)
    
    # Get top N recommendations
    _, top_indices = torch.topk(scores.squeeze(), num_recommendations)
    top_movie_indices = pairs[1][top_indices] - num_users
    
    return top_movie_indices

# 모델 학습 후 각 유저에 대한 추천 수행
node_embeddings = gat_model(data.x, data.edge_index)
for i in range(num_users):
    user_id = users['user_id'][i]
    top_movies = recommend_movies_for_user(user_id, node_embeddings, edge_index, num_recommendations=5)
    print(f"Top 5 recommended movies for user {user_id}: {top_movies}")


Epoch 0, Loss: 0.7092497944831848
Epoch 10, Loss: 9.948346814780962e-06
Epoch 20, Loss: 2.244284836097621e-12
Epoch 30, Loss: 3.4094979288629314e-20
Epoch 40, Loss: 9.269431197004435e-14
Epoch 50, Loss: 1.0993449192967485e-14
Epoch 60, Loss: 1.8111944574878726e-15
Epoch 70, Loss: 2.3613119434180674e-20
Epoch 80, Loss: 7.153816726949278e-19
Epoch 90, Loss: 1.222773374524519e-14
Top 5 recommended movies for user 1: tensor([2365, 1473, 2560, 2336, 1536])
Top 5 recommended movies for user 2: tensor([2560,    4, 9376, 5951,    3])


In [463]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch_geometric.data import Data

# 예시 유저 데이터
interactions = pd.DataFrame(columns=['user_id', 'movie_id'])
interactions['user_id'] = [1, 1, 1, 2, 2]
interactions['movie_id'] = [3, 4, 5951, 5459, 6076]
users = pd.DataFrame(columns=['user_id'])
users['user_id'] = interactions.user_id.unique()

num_users = users.shape[0]
num_movies = 9525
num_in_features = 406
num_out_features = 128  # Hidden dimension 크기 조정
num_of_heads = 1

# 영화 features
movie_features = torch.from_numpy(np.load("final_features_without_directors_0511.npy")).float()

# user indices, movie indices 생성
user_id_to_index = {user_id: idx for idx, user_id in enumerate(users['user_id'])}
movie_id_to_index = {movie_id: idx for idx, movie_id in enumerate(range(num_movies))}

# 유저 - 영화 간 상호작용 edge index로 변환
user_indices = interactions['user_id'].apply(lambda x: user_id_to_index[x])
movie_indices = interactions['movie_id'].apply(lambda x: num_users + movie_id_to_index[x])  # 유저 수만큼 offset 추가

edge_index = torch.tensor([user_indices.values, movie_indices.values], dtype=torch.long)

# 유저 초기 임베딩 생성
user_features = torch.zeros(num_users, num_in_features)
# 유저 초기 임베딩을 xavier 초기화
nn.init.xavier_uniform_(user_features)  # initialize user features with xavier initialization
# 영화 feature와 유저 feature 합치기
x = torch.cat([user_features, movie_features], dim=0)

data = Data(x=x, edge_index=edge_index)

# 유저, 영화의 임베딩 생성용 GAT
class GATLinkPredictor(nn.Module):
    def __init__(self, num_in_features, num_out_features=128, num_of_heads=1):
        super().__init__()
        self.conv1 = GATConv(num_in_features, num_out_features, heads=num_of_heads)
        self.conv2 = GATConv(num_out_features * num_of_heads, num_out_features, heads=num_of_heads)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        return x

# 유저 - 영화 간 edge를 입력받아 유저-영화 간 관계 예측
class LinkPredictor(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim * 2, 1)

    def forward(self, z, edge_index):
        row, col = edge_index
        z_row = z[row]
        z_col = z[col]
        z_concat = torch.cat([z_row, z_col], dim=1)
        return torch.sigmoid(self.linear(z_concat))

# 모델 초기화
gat_model = GATLinkPredictor(num_in_features, num_out_features, num_of_heads)
link_predictor = LinkPredictor(num_out_features * num_of_heads)

# Negative sampling
def negative_sampling(edge_index, num_users, num_movies, num_neg_samples):
    existing_edges = set(zip(edge_index[0].tolist(), edge_index[1].tolist()))
    neg_edge_index = set()
    
    while len(neg_edge_index) < num_neg_samples:
        i = torch.randint(0, num_users, (1,)).item()
        j = torch.randint(0, num_movies, (1,)).item() + num_users  # Add offset to movie indices
        if (i, j) not in existing_edges and (i, j) not in neg_edge_index:
            neg_edge_index.add((i, j))
    
    neg_edge_index = torch.tensor(list(neg_edge_index)).t()
    return neg_edge_index

# negative samples
num_neg_samples = edge_index.size(1) * 2  # Increase the number of negative samples
neg_edge_index = negative_sampling(edge_index, num_users, num_movies, num_neg_samples)

# positive, negative samples 합치기
train_edge_index = torch.cat([edge_index, neg_edge_index], dim=1)
train_labels = torch.cat([torch.ones(edge_index.size(1)), torch.zeros(neg_edge_index.size(1))])

# Train 함수 정의
def train(model, predictor, data, train_edge_index, train_labels, optimizer, epochs=200):
    model.train()
    predictor.train()
    criterion = nn.BCELoss()

    for epoch in range(epochs):
        optimizer.zero_grad()
        node_embeddings = model(data.x, data.edge_index)
        scores = predictor(node_embeddings, train_edge_index)
        loss = criterion(scores.squeeze(), train_labels)
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

# Initialize optimizer
optimizer = torch.optim.Adam(list(gat_model.parameters()) + list(link_predictor.parameters()), lr=0.005)

# 모델 학습
train(gat_model, link_predictor, data, train_edge_index, train_labels, optimizer, epochs=200)

# Function to recommend top 5 movies for a given user
def recommend_movies_for_user(user_id, node_embeddings, edge_index, num_recommendations=5):
    user_idx = torch.tensor([user_id_to_index[user_id]])
    movie_indices = torch.arange(num_users, num_users + num_movies)
    
    # Generate all possible user-movie pairs
    pairs = torch.stack([user_idx.expand(num_movies), movie_indices], dim=0)
    
    # Check existing edges to avoid recommending already liked movies
    existing_edges = edge_index[:, (edge_index[0] == user_idx)]
    existing_movies = existing_edges[1] - num_users
    
    # Remove existing edges from pairs
    mask = torch.ones(num_movies, dtype=torch.bool)
    mask[existing_movies] = False
    pairs = pairs[:, mask]
    
    # Get scores for the user-movie pairs
    scores = link_predictor(node_embeddings, pairs)
    
    # Get top N recommendations
    _, top_indices = torch.topk(scores.squeeze(), num_recommendations)
    top_movie_indices = pairs[1][top_indices] - num_users
    
    return top_movie_indices

# 모델 학습 후 각 유저에 대한 추천 수행
node_embeddings = gat_model(data.x, data.edge_index)
for i in range(num_users):
    user_id = users['user_id'][i]
    top_movies = recommend_movies_for_user(user_id, node_embeddings, edge_index, num_recommendations=5)
    print(f"Top 5 recommended movies for user {user_id}: {top_movies}")


Epoch 0, Loss: 0.6593482494354248
Epoch 10, Loss: 0.003518820507451892
Epoch 20, Loss: 1.9672154394356767e-06
Epoch 30, Loss: 8.278193064370498e-08
Epoch 40, Loss: 2.8561521148162683e-08
Epoch 50, Loss: 6.0337401741605845e-09
Epoch 60, Loss: 1.0458835575377634e-08
Epoch 70, Loss: 1.3555521150010463e-07
Epoch 80, Loss: 6.5999681275741295e-09
Epoch 90, Loss: 6.255229578755461e-08
Epoch 100, Loss: 7.21210042797793e-08
Epoch 110, Loss: 4.2921638510051707e-07
Epoch 120, Loss: 3.744747179013075e-09
Epoch 130, Loss: 1.0258002447471881e-08
Epoch 140, Loss: 1.0801806560323257e-08
Epoch 150, Loss: 1.3551009203638387e-07
Epoch 160, Loss: 1.0430602159772207e-08
Epoch 170, Loss: 2.6606508995996592e-08
Epoch 180, Loss: 1.650311887146927e-08
Epoch 190, Loss: 1.4418660043702403e-07
Top 5 recommended movies for user 1: tensor([6076, 5459, 1172, 9263, 1963])
Top 5 recommended movies for user 2: tensor([5951,    4,    3, 1172, 9263])


# Check

In [360]:
node_embeddings

tensor([[-0.6118, -0.0000, -0.0000,  ..., -0.0000,  0.0000,  1.1427],
        [-0.4571, -0.0000, -0.5555,  ..., -0.0000,  0.9184,  0.0000],
        [-0.3924, -0.7799, -0.0000,  ..., -0.0000,  0.9841,  1.1503],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000, -0.0000, -1.5375],
        [ 0.0000,  2.9648,  0.0000,  ...,  0.0000, -0.0000, -0.0000],
        [ 1.4215,  0.0000,  0.0000,  ...,  0.0000,  0.0000, -0.9914]],
       grad_fn=<MulBackward0>)

In [47]:
movie_metadata = pd.read_excel("cinemate_data_0511.xlsx")
movie_metadata

,movie_id,rating,backdrop_path,original_title,title,release_date,poster_path,overview
0,1146410,0.000,/u3JuBKQTuyfvWy7Rm01Ugej6Qs5.jpg,여행자의 필요,여행자의 필요,2024-04-24,/fNhyJp2fy47vg66LZ8aA2b0tsVQ.jpg,"어디서 온지 모르는 이 사람은 불란서에서 왔다고 하고, 어린애 피리를 근린공원에서 ..."
1,876127,6.800,/obJgmXo6wYEYs8OuIvRgYhx7tHx.jpg,도그데이즈,도그데이즈,2024-02-07,/bl3YQHEENj4B80Ku3PkTJWWLnj5.jpg,깔끔한 성격의 계획형 싱글남 민상. 영끌까지 모아 산 건물을 개똥밭으로 만드는 세입...
2,1072342,5.729,/klb3yODwKMRle19EcvdeSatEZL7.jpg,Night Swim,나이트 스윔,2024-01-03,/4Q20D0HCYNHwvTna4OXJNCDvyI3.jpg,넓은 수영장이 있는 새집으로 이사 온 후 더할 나위 없는 행복을 느끼고 있는 ‘레이...
3,838209,7.500,/aINel9503ompOlGKn4sIVMg09Un.jpg,파묘,파묘,2024-02-22,/tw0i3kkmOTjDjGFZTLHKhoeXVvA.jpg,"미국 LA, 거액의 의뢰를 받은 무당 화림과 봉길은 기이한 병이 대물림되는 집안의 ..."
4,1017163,8.600,/3DPJtqKKMl3Lv8rMxwqNLlRc947.jpg,범죄도시 4,범죄도시 4,2024-04-24,/h1YarEjeYurkAwXgfY1RDMVCiin.jpg,"신종 마약 사건 3년 뒤, 괴물형사 마석도와 서울 광수대는 배달앱을 이용한 마약 판..."
...,...,...,...,...,...,...,...,...
9520,67319,6.500,/3Ztq49qcwnXSC9JFmTuaLNYEGKX.jpg,So This Is Paris,여기는 파리,1926-07-31,/bVcYRrzmA95SIlPVd1RGGrd17XA.jpg,폴과 수자나 부부는 조용한 마을에서 행복하게 살고 있다. 그러던 중 화려한 댄서들이...
9521,5991,7.785,/sTyt7e2PB5nfY02reL1iZUYPbly.jpg,Der letzte Mann,마지막 웃음,1924-12-23,/woR2d4zkWgCPlvwQ9kMHcrdiGPJ.jpg,은발의 멋진 구렛나루 수염을 쓰다듬으며 금빛 단추의 제복을 뽐내는 한 호텔 도어맨이...
9522,28974,6.731,/y8WOKk49VdiA4RZ1UeFCt8xPVTF.jpg,A Woman of Paris: A Drama of Fate,파리의 여인,1923-10-01,/s24VgyvJliT7fUA909ZHJpoBx1D.jpg,마리(Marie St. Clair: 에드나 퍼비안스 분)는 계부의 반대를 피해 애인...
9523,175472,5.800,/bKNfpShviRHQAFwruRpqjj6iqxC.jpg,Vier um die Frau,여자를 둘러싼 넷,1921-02-03,/83nTPJQnzrn78ppVf5L5CEOIkIg.jpg,해리 유켐은 그의 사랑하는 아내에게 가짜와 장물을 거래하는 지하 세계의 중간거래상들...


In [53]:
final_metadata = pd.read_excel("final_metadata_with_overview_embeddings_ver5_0511.xlsx")
final_metadata

,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,directors,ordering,...,embedding_374,embedding_375,embedding_376,embedding_377,embedding_378,embedding_379,embedding_380,embedding_381,embedding_382,embedding_383
0,tt31015344,A Traveler's Needs,Yeohaengjaui Pilyo,2024,90,Drama,6.5,75,nm0393254,1,...,0.016458,-0.023006,-0.050298,0.037739,-0.076090,-0.010432,-0.058701,0.017649,0.035267,-0.023228
1,tt30911335,Dog Days,Dogeudeijeu,2024,120,Drama,7.8,31,nm15730187,1,...,-0.034701,-0.008065,-0.024117,-0.003312,0.038821,-0.030186,0.019926,0.074339,-0.018989,0.055851
2,tt9682428,Night Swim,Night Swim,2024,98,"Horror,Thriller",4.7,12654,nm3241399,1,...,0.003738,-0.049932,-0.121888,-0.054388,-0.020564,0.050116,0.021484,0.052849,0.035944,0.025606
3,tt27802490,Exhuma,Pamyo,2024,134,"Horror,Mystery,Thriller",7.4,495,nm6940206,1,...,0.051392,-0.002096,0.044035,0.059909,-0.044593,-0.032423,0.081002,0.020807,0.021268,0.008938
4,tt27811040,The Roundup: Punishment,Beomjoedosi4,2024,109,"Action,Crime,Thriller",8.8,101,nm2441168,10,...,0.041042,-0.039577,0.034051,0.041895,-0.132000,-0.051895,0.089292,-0.132800,-0.011520,-0.006363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9520,tt0017409,So This Is Paris,So This Is Paris,1926,60,Comedy,7.1,746,nm0523932,1,...,-0.052198,-0.056215,-0.065513,0.068392,0.040504,-0.013506,-0.068042,0.001189,-0.021429,-0.103460
9521,tt0015064,The Last Laugh,Der letzte Mann,1924,88,Drama,8.0,15105,nm0003638,1,...,0.029271,-0.052846,0.032112,0.051352,-0.038684,-0.013877,0.074433,-0.020061,-0.008468,0.026134
9522,tt0014624,A Woman of Paris: A Drama of Fate,A Woman of Paris: A Drama of Fate,1923,82,"Drama,Romance",6.9,6104,nm0000122,10,...,0.018064,-0.050176,-0.045593,0.036943,-0.046362,-0.033189,-0.061279,-0.036766,0.073398,-0.105024
9523,tt0012806,Four Around the Woman,Vier um die Frau,1921,52,Drama,5.9,581,nm0000485,1,...,-0.047176,-0.100745,0.056739,0.063528,-0.031887,-0.028223,-0.002628,-0.041470,0.026425,0.023838


In [471]:
# Top 5 recommended movies for user 1: tensor([130,  16, 184,  92,   7])
# Top 5 recommended movies for user 2: tensor([121,   4, 161, 109, 107])
# Top 5 recommended movies for user 3: tensor([ 16,  71, 130, 109,   4])
# [504, 529, 628, 616, 307]
# tensor([9148, 9207, 3478, 2557, 8102])
# ([ 2,  7, 31, 25, 23])  [5459, 6076]
# ([8684, 5259, 6462, 1362, 8183])
# ([5321, 6482, 9439, 3483, 4080])
# [ 899,  672, 2028, 1151, 1233]
# tensor([3355, 2098, 3049,    5, 8588])
# Top 5 recommended movies for user 1: tensor([2365, 1473, 2560, 2336, 1536])
# Top 5 recommended movies for user 2: tensor([2560,    4, 9376, 5951,    3])
# Top 5 recommended movies for user 1: tensor([6076, 5459, 1172, 9263, 1963])
# Top 5 recommended movies for user 2: tensor([5951,    4,    3, 1172, 9263])
final_metadata.iloc[9263].genres

'Fantasy'

In [472]:
movie_metadata.iloc[9263].overview

nan

In [386]:
movie_metadata.iloc[8102]

movie_id                                    416101
rating                                         5.6
backdrop_path     /oYBtXawjNhevCkIT8Na28gAsLPZ.jpg
original_title                      Ninja Avengers
title                               Ninja Avengers
release_date                            1987-01-01
poster_path       /4viIQgcqj9e8RD09khPg1LoE9iL.jpg
overview                                       NaN
Name: 8102, dtype: object

In [387]:
movie_metadata.iloc[8102].overview

nan

In [95]:
movie_metadata.iloc[9209]

movie_id                                                      42787
rating                                                          6.2
backdrop_path                      /aTmbTmoXqc85rFtKc6n12XwWUaT.jpg
original_title                                       Lady in a Cage
title                                                       새장속의 여인
release_date                                             1964-06-10
poster_path                        /f0eafb0uhYhBTvA2MRB1vN0yJL7.jpg
overview          자신의 집에서 실내 엘리베이터에 갇혀 버린 후, 집안에 불법 침입한 괴한들로부터 위...
Name: 9209, dtype: object

In [131]:
final_metadata.iloc[8954]

tconst                tt0066842
primaryTitle      Bleak Moments
originalTitle     Bleak Moments
startYear                  1971
runtimeMinutes              111
                      ...      
embedding_379          0.078274
embedding_380          0.050355
embedding_381          0.022296
embedding_382         -0.012955
embedding_383          0.002175
Name: 8954, Length: 396, dtype: object

In [ ]:
# [3, 4, 5951]를 좋아하는 사용자에게 추천하는 영화
# array([9209, 2630, 8562, 8954, 5206]) 첫번째 추천 결과
# array([8562, 5206, 9209, 5723, 8954]) 두번째 추천 결과
# [8562 8711 5723 7577 9209]

In [138]:
final_metadata.iloc[326].genres

'Drama,Romance'

In [137]:
movie_metadata.iloc[326].overview

'연인이 된 ‘비비안’과 ‘로이’는 미래를 함께 하기 위해, ‘로이’가 상속 받은 오래된 저택을 매각하기 위해 시칠리아에 간다. 바쁜 ‘로이’로 인해 혼자 남은 ‘비비안’은 우연히 ‘안나’를 치료해주고 친해진다. ‘로이’는 그런 ‘비비안’에게 마치 알고 있던 사람처럼 ‘안나’를 조심하라고 한다. ‘로이’의 행동이 과민반응이라고 생각하던 그때, ‘안나’에게서 ‘로이’와 똑같은 타투를 발견하게 되는데…'

In [219]:
final_metadata.iloc[5951].genres

'Horror,Thriller'

In [215]:
movie_metadata.iloc[5957].overview

nan

In [218]:
movie_metadata.iloc[5951]

movie_id                                                      13510
rating                                                        6.826
backdrop_path                      /5h2nljNHxqpXSs5DrNjXUVHX83F.jpg
original_title                                            Eden Lake
title                                                        이든 레이크
release_date                                             2008-09-12
poster_path                        /wwLX6RPLZlchZWzEBZ2mTQ9Nooh.jpg
overview          둘만의 오붓한 시간을 보내고자 시골의 조용한 호수로 주말여행을 떠난 제니와 스티브....
Name: 5951, dtype: object